# Model Selection

In [1]:
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_parquet('datasets/featured/featured.parquet')

In [3]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Season,LotRatio
0,60.00,RL,65.00,8450.00,0,Reg,Lvl,0,Inside,Gtl,CollgCr,Norm,0,1Fam,2Story,7.00,5.00,2003.00,2003.00,Gable,0,VinylSd,VinylSd,196.00,Gd,TA,PConc,Gd,TA,No,GLQ,706.00,Unf,0.00,150.00,856.00,0,Ex,1,SBrkr,856.00,854.00,0.00,1710.00,1.00,0.00,2.00,1.00,3.00,1.00,Gd,8.00,Typ,0.00,Attchd,2003.00,RFn,2.00,548.00,TA,TA,Y,0.00,61.00,0.00,0.00,0.00,0.00,0.00,2.00,2008.00,WD,Normal,208500.00,Winter,130.00
1,20.00,RL,80.00,9600.00,0,Reg,Lvl,0,FR2,Gtl,Rare,Feedr,0,1Fam,1Story,6.00,8.00,1976.00,1976.00,Gable,0,MetalSd,MetalSd,0.00,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.00,Unf,0.00,284.00,1262.00,0,Ex,1,SBrkr,1262.00,0.00,0.00,1262.00,0.00,1.00,2.00,0.00,3.00,1.00,TA,6.00,Typ,1.00,Attchd,1976.00,RFn,2.00,460.00,TA,TA,Y,298.00,0.00,0.00,0.00,0.00,0.00,0.00,5.00,2007.00,WD,Normal,181500.00,Spring,120.00
2,60.00,RL,68.00,11250.00,0,IR1,Lvl,0,Inside,Gtl,CollgCr,Norm,0,1Fam,2Story,7.00,5.00,2001.00,2002.00,Gable,0,VinylSd,VinylSd,162.00,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.00,Unf,0.00,434.00,920.00,0,Ex,1,SBrkr,920.00,866.00,0.00,1786.00,1.00,0.00,2.00,1.00,3.00,1.00,Gd,6.00,Typ,1.00,Attchd,2001.00,RFn,2.00,608.00,TA,TA,Y,0.00,42.00,0.00,0.00,0.00,0.00,0.00,9.00,2008.00,WD,Normal,223500.00,Fall,165.44
3,70.00,RL,60.00,9550.00,0,IR1,Lvl,0,Corner,Gtl,Crawfor,Norm,0,1Fam,2Story,7.00,5.00,1915.00,1970.00,Gable,0,Wd Sdng,Wd Shng,0.00,TA,TA,BrkTil,TA,Gd,No,ALQ,216.00,Unf,0.00,540.00,756.00,0,Gd,1,SBrkr,961.00,756.00,0.00,1717.00,1.00,0.00,1.00,0.00,3.00,1.00,Gd,7.00,Typ,1.00,Detchd,1998.00,Unf,3.00,642.00,TA,TA,Y,0.00,35.00,272.00,0.00,0.00,0.00,0.00,2.00,2006.00,WD,Abnorml,140000.00,Winter,159.17
4,60.00,RL,84.00,14260.00,0,IR1,Lvl,0,FR2,Gtl,NoRidge,Norm,0,1Fam,2Story,8.00,5.00,2000.00,2000.00,Gable,0,VinylSd,VinylSd,350.00,Gd,TA,PConc,Gd,TA,Av,GLQ,655.00,Unf,0.00,490.00,1145.00,0,Ex,1,SBrkr,1145.00,1053.00,0.00,2198.00,1.00,0.00,2.00,1.00,4.00,1.00,Gd,9.00,Typ,1.00,Attchd,2000.00,RFn,3.00,836.00,TA,TA,Y,192.00,84.00,0.00,0.00,0.00,0.00,0.00,12.00,2008.00,WD,Normal,250000.00,None,169.76


## Models

In [10]:
from _utils._sklearn_models import *

In [5]:
X, y = prepare_dataframe(df, 'SalePrice')

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
reg_models = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)

In [8]:
models, predictions = reg_models.fit(X_train, X_test, y_train, y_test)

100%|██████████| 42/42 [00:39<00:00,  1.06it/s]


In [9]:
models

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
XGBRegressor,0.77,0.91,21169.51,0.30
GradientBoostingRegressor,0.74,0.89,22560.96,0.76
ExtraTreesRegressor,0.70,0.88,23970.29,1.71
LGBMRegressor,0.65,0.86,26061.05,0.19
HistGradientBoostingRegressor,0.62,0.85,26909.28,2.48
RandomForestRegressor,0.61,0.84,27526.57,2.41
BaggingRegressor,0.58,0.83,28554.94,0.27
TweedieRegressor,0.43,0.77,33089.85,0.03
AdaBoostRegressor,0.42,0.77,33330.12,0.37


# XGBRegressor

In [11]:
model = XGBRegressorModel(X, y)

In [12]:
model.init_model()

In [13]:
model.get_scores()

,0,1
5,neg_mean_squared_error,736259917.1954353
6,neg_root_mean_squared_error,26981.906091150693
7,r2,0.8768760807812118


In [15]:
model.model.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'gpu_id': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

## Hyperparameter Tunning

In [16]:
from sklearn.model_selection import GridSearchCV

In [17]:
hyperparameters = {
    'n_estimators': [100, 400, 800],
    'max_depth': [None, 3, 6, 9, 12, 15],
    'learning_rate': [None, 0.1, 0.01, 0.001],
    'min_child_weight': [None, 1, 10, 100],
    "colsample_bytree": [None, 0.5, 0.7, 1]
    }

In [19]:
xgb_best_grid = GridSearchCV(model.model, hyperparameters, cv=5, n_jobs=-1, verbose=True).fit(X, y)

Fitting 5 folds for each of 1152 candidates, totalling 5760 fits


In [20]:
xgb_best_grid.best_score_

0.893293526789502

In [21]:
xgb_best_grid.best_params_

{'colsample_bytree': None,
 'learning_rate': 0.1,
 'max_depth': 3,
 'min_child_weight': None,
 'n_estimators': 400}

In [22]:
hyperparameters = {
    'n_estimators': [200, 300, 400, 500, 600],
    'max_depth': [2, 3, 4, 5],
    'learning_rate': [0.1],
    "colsample_bytree": [None, 0.4, 0.5, 0.6]
    }

In [24]:
xgb_best_grid = GridSearchCV(model.model, hyperparameters, cv=5, n_jobs=-1, verbose=True).fit(X, y)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


In [25]:
xgb_best_grid.best_score_

0.8943929489321315

In [26]:
xgb_best_grid.best_params_

{'colsample_bytree': None,
 'learning_rate': 0.1,
 'max_depth': 4,
 'n_estimators': 300}

In [37]:
hyperparameters = {
 'colsample_bytree': None,
 'learning_rate': 0.1,
 'max_depth': 4,
 'n_estimators': 300}

In [38]:
model.model.set_params(**hyperparameters).fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=300, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [39]:
model.get_scores()

,0,1
5,neg_mean_squared_error,635223228.7587614
6,neg_root_mean_squared_error,25013.093953597978
7,r2,0.8943929489321315
